In [6]:
import pandas as pd
import numpy as np
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import altair as alt
import dash_html_components as html
import json

# Reading in the csv
df = pd.read_csv("../data/raw/HIV.csv", low_memory=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40540 entries, 0 to 40539
Data columns (total 22 columns):
 #   Column                                                                          Non-Null Count  Dtype  
---  ------                                                                          --------------  -----  
 0   DATAFLOW                                                                        40540 non-null  object 
 1   REF_AREA:Geographic area                                                        40540 non-null  object 
 2   INDICATOR:Indicator                                                             40540 non-null  object 
 3   SEX:Sex                                                                         40540 non-null  object 
 4   TIME_PERIOD:Time period                                                         40540 non-null  object 
 5   OBS_VALUE:Observation Value                                                     40540 non-null  object 
 6   UNIT_MULTIPLIE

Pivot the dataframe to have each indicator on seperate column and remove the string before the column symbol on each column name \
Since we are only going to look at the full picture of each country in terms of gender, we will only keep the rows where the Sex column equals to Total.

In [ ]:
df_new = df.pivot_table(values='OBS_VALUE:Observation Value', index=['REF_AREA:Geographic area', 'SEX:Sex', 'TIME_PERIOD:Time period', 'UNIT_MEASURE:Unit of measure'], columns='INDICATOR:Indicator', dropna=True, aggfunc='first')
df_new.reset_index(inplace=True)
df_new.columns = df_new.columns.str.split(':').str[-1].str.strip()
df_new['Geographic area'] = df_new['Geographic area'].str.split(':').str[-1].str.strip()
df_new['Sex'] = df_new['Sex'].str.split(':').str[-1].str.strip()
df_new['Unit of measure'] = df_new['Unit of measure'].str.split(':').str[-1].str.strip()
df_new = df_new[df_new['Sex'] == 'Total']

Check if there are any empty columns 

In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13892 entries, 94 to 27616
Data columns (total 21 columns):
 #   Column                                                                                                                                                                     Non-Null Count  Dtype 
---  ------                                                                                                                                                                     --------------  ----- 
 0   Geographic area                                                                                                                                                            13892 non-null  object
 1   Sex                                                                                                                                                                        13892 non-null  object
 2   Time period                                                                                   

Since column 16-20 are almost fully empty, we will remove them from the dataframe 

In [ ]:
df_new = df_new.iloc[:, 0:16]

Update the data types for variables

In [ ]:
# Convert time period from string to datetime
df_new['Time period'] = pd.to_datetime(df_new['Time period'], format='%Y').dt.year

# Convert fully numeric indicators into numeric type 
def is_numeric(column):
    try:
        pd.to_numeric(column)
        return True
    except ValueError:
        return False
numeric_columns = df_new.iloc[:, 4:16].apply(is_numeric)
numeric_column_names = numeric_columns[numeric_columns].index
df_new[numeric_column_names] = df_new[numeric_column_names].apply(pd.to_numeric)
df_new.info()
# df_new.to_csv("dash.csv")

<class 'pandas.core.frame.DataFrame'>
Index: 13892 entries, 94 to 27616
Data columns (total 16 columns):
 #   Column                                                                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                                                                           --------------  -----  
 0   Geographic area                                                                                                                                                  13892 non-null  object 
 1   Sex                                                                                                                                                              13892 non-null  object 
 2   Time period                                                                                                                       

In [ ]:
df_new.drop('Unit of measure', axis=1, inplace=True)

# Step 2: Aggregate the data for each 'Geographic area', 'Sex', and 'Time period'
# We will take the first non-null value for each group
aggregation_functions = {col: 'first' for col in df_new.columns[3:]}
df_aggregated = df_new.groupby(['Geographic area', 'Sex', 'Time period'], as_index=False).agg(aggregation_functions)

# Drop any remaining rows with all NaN values in the indicator columns
df_aggregated.dropna(subset=df_aggregated.columns[3:], how='all', inplace=True)

# Reset index to tidy up the DataFrame
df_aggregated.reset_index(drop=True, inplace=True)



In [ ]:
# Display the transformed DataFrame
df_aggregated

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Tabs([
        ### First tab
        dbc.Tab([ 
            html.H1('HIV Indicator Trends by Country and Year'),
            html.P('Select an indicator and up to 4 countries to compare their trends over time.'),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='indicator-dropdown',
                        value=df_aggregated.columns[3],  # Default value is the first indicator column
                        options=[{'label': col, 'value': col} for col in df_aggregated.columns[3:]], 
                        placeholder='Choose 1 indicator...'),
                    dcc.Dropdown(
                        id='country-dropdown',
                        options=[{'label': country, 'value': country} for country in df_aggregated['Geographic area'].unique()],
                        placeholder='Choose up to 4 countries...',
                        multi=True)  # Allow multiple selections
                ]),
            ]),
            dbc.Row([
                html.Iframe(id='trend-chart', style={'border-width': '0', 'width': '100%', 'height': '400px'}),
                dcc.RangeSlider(
                        id='year-slider',
                        min=df_aggregated['Time period'].min(),
                        max=df_aggregated['Time period'].max(),
                        marks={str(year): str(year) for year in range(df_aggregated['Time period'].min(), df_aggregated['Time period'].max() + 1)},
                        step=1,
                        value=[df_aggregated['Time period'].min(), df_aggregated['Time period'].max()]
                )
            ])
        ], label='Indicator Trend'),
        ### Second tab
        dbc.Tab([html.H1('Indicator map'), 
                 html.P(" ")], 
                 label='Indicator map'),
        ### Third tab
        dbc.Tab([html.H1('Indicator summary statistics'),
                 html.P(" ")], 
                 label='Indicator summary statistics')
        ])
])

# Callback for updating the chart based on selections
@app.callback(
    Output('trend-chart', 'srcDoc'),
    [Input('indicator-dropdown', 'value'),
     Input('country-dropdown', 'value'),
     Input('year-slider', 'value')]
)
def update_chart(selected_indicator, selected_countries, selected_years):
    if selected_countries is None or selected_indicator is None or len(selected_countries) > 4:
        return 'Please select an indicator and up to 4 countries.'

    # Filter based on the selected years and countries
    chart_df = df_aggregated[df_aggregated['Time period'].between(*selected_years)]
    chart_df = chart_df[chart_df['Geographic area'].isin(selected_countries)]
    
    # Create the Altair chart
    base = alt.Chart(chart_df).encode(
        x=alt.X('Time period:O', axis=alt.Axis(title='Year')),
        y=alt.Y(f"{selected_indicator}:Q", axis=alt.Axis(title=selected_indicator)),
        color='Geographic area:N'
    )
    
    line_chart = base.mark_line(point=True).properties(
        width=700,
        height=400
    )
    
    return line_chart.to_html()

if __name__ == '__main__':
    app.run_server(debug=True)